In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
pd.options.display.max_rows = 5
import matplotlib.pyplot as plt
import gvar as gv
import fkfpi_lib as flib

In [2]:
switches = dict()
switches['ensemble'] = [
    'a15m310','a12m310','a09m310',
    'a15m220','a12m220','a09m220','a12m220S','a12m220L',
    'a15m130','a12m130']

switches['ansatz'] = dict()
switches['ansatz']['FV'] = True # True turns on NLO FV correction
switches['ansatz']['type'] = 'MA' # Type of fit: 'xpt' or 'MA'
switches['scale'] = 'PK' # PP, PK, KK, LamChi = 4 * pi * sqrt(FA * FB)

In [3]:
# Set priors for Xpt
priors = dict()
priors['L5'] = gv.gvar(0.0, 1.0)
priors['s2'] = gv.gvar(0.0, 1.0)
# Physical parameters from PDG
phys_p = dict()
# http://pdg.lbl.gov/2016/tables/rpp2016-tab-mesons-light.pdf
phys_p['mpi'] = gv.gvar(134.8, 0.3) #FLAG 2017 (16)
phys_p['mka'] = gv.gvar(494.2 , 0.3) #FLAG 2017 (16)
phys_p['F0'] = gv.gvar(80,20) #FLAG use of F0 in SU(2) correction for FK/Fpi
phys_p['meta'] = gv.gvar(547.862,0.017) #PDG
# http://pdg.lbl.gov/2015/reviews/rpp2015-rev-pseudoscalar-meson-decay-cons.pdf
phys_p['Fpi'] = gv.gvar(130.2/np.sqrt(2), 1.7/np.sqrt(2)) #PDG fpi+ eq(16)
phys_p['Fka'] = gv.gvar(155.6/np.sqrt(2), 0.4/np.sqrt(2)) #PDG fK++ eq(16)
if switches['scale'] == 'PK':
    phys_p['Lchi'] = 4*np.pi*np.sqrt(phys_p['Fpi']*phys_p['Fka'])
elif switches['scale'] == 'PP':
    phys_p['Lchi'] = 4*np.pi*phys_p['Fpi']
elif switches['scale'] == 'KK':
    phys_p['Lchi'] = 4*np.pi*phys_p['Fka']
# FLAG FK/Fpi
flag_FKFpi = gv.gvar(1.1933, 0.0029)

In [4]:
data = pd.read_csv('./github_fkfpi_v1.csv')
hisq_params = pd.read_csv('./hisq_params.csv')

In [5]:
fmt_data = flib.format_data(switches,data,hisq_params,priors)
print(fmt_data)

{'y': array([1.0955(14), 1.0998(15), 1.1088(16), 1.1257(18), 1.1299(21),
       1.1430(17), 1.1306(27), 1.1275(28), 1.1713(41), 1.1598(35)], dtype=object), 'x': {'Lchi': array([ 0.99302805,  0.81068306,  0.60252409,  0.96968323,  0.78381563,
        0.57536441,  0.78544483,  0.78404416,  0.96388758,  0.77102255]), 'mpiL': array([ 3.77737891,  4.52684875,  4.5036145 ,  3.97023563,  4.29651393,
        4.69899125,  3.25337141,  5.36074159,  3.22977757,  3.90131993])}, 'p': {'mka': array([0.40460(25), 0.32416(21), 0.24110(14), 0.38691(21), 0.31003(17),
       0.228703(91), 0.31044(22), 0.31018(19), 0.37796(23), 0.30204(22)], dtype=object), 'a2di': array([0.087(11), 0.0312(26), 0.00515(42), 0.086(10), 0.0300(25),
       0.00496(40), 0.0309(26), 0.0303(25), 0.084(10), 0.0304(25)], dtype=object), 's2': 0.0(1.0), 'mss': array([0.51875(17), 0.41803(16), 0.31170(11), 0.51495(14), 0.41539(12),
       0.309358(70), 0.41576(15), 0.41557(12), 0.51314(13), 0.41420(11)], dtype=object), 'aw0': array([

In [6]:
fit = flib.fit_data(switches,fmt_data,phys_p)

In [7]:
print(fit)

Least Square Fit:
  chi2/dof [dof] = 1.5 [10]    Q = 0.13    logGBF = 30.734

Parameters:
          mka 0    0.40462 (24)      [  0.40460 (25) ]  
              1    0.32417 (19)      [  0.32416 (21) ]  
              2    0.24097 (13)      [  0.24110 (14) ]  
              3    0.38698 (20)      [  0.38691 (21) ]  
              4    0.31007 (16)      [  0.31003 (17) ]  
              5   0.228679 (87)      [ 0.228703 (91) ]  
              6    0.31046 (21)      [  0.31044 (22) ]  
              7    0.31033 (17)      [  0.31018 (19) ]  
              8    0.37788 (23)      [  0.37796 (23) ]  
              9    0.30210 (20)      [  0.30204 (22) ]  
         a2di 0      0.086 (10)      [    0.087 (11) ]  
              1     0.0311 (26)      [   0.0312 (26) ]  
              2    0.00518 (42)      [  0.00515 (42) ]  
              3      0.083 (10)      [    0.086 (10) ]  
              4     0.0297 (25)      [   0.0300 (25) ]  
              5    0.00498 (40)      [  0.00496 (40) ] 

In [8]:
print(flib.fkfpi_phys(phys_p,fit))

prediction from LQCD
FK / Fpi =  1.1875(14)
FK+/Fpi+ =  1.1852(15)
prediction from LQCD + meta_PDG
FK / Fpi =  1.1846(15)
FK+/Fpi+ =  1.1822(15)
None
